# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 7094, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 7094 (delta 118), reused 7 (delta 2), pack-reused 6822 (from 3)
Receiving objects: 100% (7094/7094), 415.78 MiB | 37.02 MiB/s, done.
Resolving deltas: 100% (3849/3849), done.
Updating files: 100% (506/506), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 28.1 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/SLIM'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIM_BPR',
    'n_folds': 5,
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIM_BPR_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

def objective_function_SLIM_BPR(optuna_trial):

    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 500),
                    "epochs": optuna_trial.suggest_int("epochs", 1, 50),
                    "lambda_i": optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True),
                    "lambda_j": optuna_trial.suggest_float("lambda_j", 1e-5, 1e-1, log=True),
                    "learning_rate": optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
                }
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = SLIM_BPR_Python(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIM_BPR, timeout=11*3600)

[I 2025-01-06 23:12:21,536] Using an existing study with name 'hyperparameters_tuning_SLIM_BPR_MAP' instead of creating a new one.


Epoch 1, Iteration 35736 in 9.73 seconds. Samples per second 3671.65
Epoch 2, Iteration 35736 in 7.15 seconds. Samples per second 4994.73
Epoch 3, Iteration 35736 in 5.74 seconds. Samples per second 6221.60
Epoch 4, Iteration 35736 in 4.92 seconds. Samples per second 7270.18
Epoch 5, Iteration 35736 in 4.34 seconds. Samples per second 8237.23
Epoch 6, Iteration 35736 in 3.97 seconds. Samples per second 9007.49
Epoch 7, Iteration 35736 in 3.81 seconds. Samples per second 9372.87
Epoch 8, Iteration 35736 in 3.52 seconds. Samples per second 10155.08
Epoch 9, Iteration 35736 in 3.39 seconds. Samples per second 10556.69
Epoch 10, Iteration 35736 in 3.32 seconds. Samples per second 10773.66
Epoch 11, Iteration 35736 in 3.20 seconds. Samples per second 11163.62
Epoch 12, Iteration 35736 in 3.13 seconds. Samples per second 11417.49
Epoch 13, Iteration 35736 in 3.13 seconds. Samples per second 11426.60
Epoch 14, Iteration 35736 in 3.05 seconds. Samples per second 11717.80
Epoch 15, Iteration 35

[I 2025-01-06 23:38:11,534] Trial 61 finished with value: 0.041418588642201215 and parameters: {'topK': 15, 'epochs': 44, 'lambda_i': 0.014092591080366347, 'lambda_j': 0.002423998475823978, 'learning_rate': 0.0966264397706937}. Best is trial 54 with value: 0.04209128232743534.


Epoch 1, Iteration 35736 in 10.32 seconds. Samples per second 3463.61
Epoch 2, Iteration 35736 in 7.55 seconds. Samples per second 4734.95
Epoch 3, Iteration 35736 in 5.84 seconds. Samples per second 6115.40
Epoch 4, Iteration 35736 in 4.90 seconds. Samples per second 7295.33
Epoch 5, Iteration 35736 in 7.19 seconds. Samples per second 4973.64
Epoch 6, Iteration 35736 in 5.31 seconds. Samples per second 6723.93
Epoch 7, Iteration 35736 in 3.94 seconds. Samples per second 9069.22
Epoch 8, Iteration 35736 in 3.75 seconds. Samples per second 9520.58
Epoch 9, Iteration 35736 in 3.56 seconds. Samples per second 10049.40
Epoch 10, Iteration 35736 in 3.43 seconds. Samples per second 10420.61
Epoch 11, Iteration 35736 in 3.35 seconds. Samples per second 10671.44
Epoch 12, Iteration 35736 in 3.24 seconds. Samples per second 11034.70
Epoch 13, Iteration 35736 in 3.15 seconds. Samples per second 11360.39
Epoch 14, Iteration 35736 in 3.13 seconds. Samples per second 11416.47
Epoch 15, Iteration 35

[I 2025-01-07 00:04:14,198] Trial 62 finished with value: 0.042585162596466386 and parameters: {'topK': 12, 'epochs': 44, 'lambda_i': 0.044831371509862585, 'lambda_j': 0.0024480685022516715, 'learning_rate': 0.09908477965723278}. Best is trial 62 with value: 0.042585162596466386.


...

[I 2025-01-07 09:25:44,443] Trial 83 finished with value: 0.0402086874728101 and parameters: {'topK': 13, 'epochs': 45, 'lambda_i': 0.020697779626470462, 'lambda_j': 0.0006105540730717701, 'learning_rate': 0.056114819154508215}. Best is trial 62 with value: 0.042585162596466386.


Epoch 1, Iteration 35736 in 9.28 seconds. Samples per second 3848.92
Epoch 2, Iteration 35736 in 6.82 seconds. Samples per second 5236.27
Epoch 3, Iteration 35736 in 5.51 seconds. Samples per second 6482.77
Epoch 4, Iteration 35736 in 4.81 seconds. Samples per second 7428.43
Epoch 5, Iteration 35736 in 4.21 seconds. Samples per second 8485.48
Epoch 6, Iteration 35736 in 3.89 seconds. Samples per second 9176.26
Epoch 7, Iteration 35736 in 3.63 seconds. Samples per second 9842.71
Epoch 8, Iteration 35736 in 3.49 seconds. Samples per second 10228.71
Epoch 9, Iteration 35736 in 3.33 seconds. Samples per second 10724.68
Epoch 10, Iteration 35736 in 3.23 seconds. Samples per second 11050.03
Epoch 11, Iteration 35736 in 3.21 seconds. Samples per second 11140.94
Epoch 12, Iteration 35736 in 3.12 seconds. Samples per second 11451.27
Epoch 13, Iteration 35736 in 3.15 seconds. Samples per second 11340.42
Epoch 14, Iteration 35736 in 3.09 seconds. Samples per second 11565.98
Epoch 15, Iteration 35

[I 2025-01-07 09:53:22,800] Trial 84 finished with value: 0.038624454209923845 and parameters: {'topK': 26, 'epochs': 46, 'lambda_i': 0.0028012963201345495, 'lambda_j': 0.06723574712758648, 'learning_rate': 0.048408917534310425}. Best is trial 62 with value: 0.042585162596466386.


Epoch 1, Iteration 35736 in 9.31 seconds. Samples per second 3839.98
Epoch 2, Iteration 35736 in 6.86 seconds. Samples per second 5210.92
Epoch 3, Iteration 35736 in 5.52 seconds. Samples per second 6473.94
Epoch 4, Iteration 35736 in 4.77 seconds. Samples per second 7497.97
Epoch 5, Iteration 35736 in 4.24 seconds. Samples per second 8434.43
Epoch 6, Iteration 35736 in 3.95 seconds. Samples per second 9055.06
Epoch 7, Iteration 35736 in 3.64 seconds. Samples per second 9807.30
Epoch 8, Iteration 35736 in 3.46 seconds. Samples per second 10314.17
Epoch 9, Iteration 35736 in 3.39 seconds. Samples per second 10548.96
Epoch 10, Iteration 35736 in 3.31 seconds. Samples per second 10791.61
Epoch 11, Iteration 35736 in 3.16 seconds. Samples per second 11316.16
Epoch 12, Iteration 35736 in 3.15 seconds. Samples per second 11332.30
Epoch 13, Iteration 35736 in 3.08 seconds. Samples per second 11591.47
Epoch 14, Iteration 35736 in 3.06 seconds. Samples per second 11695.67
Epoch 15, Iteration 35

[I 2025-01-07 10:19:28,824] Trial 85 finished with value: 0.0002664826418719655 and parameters: {'topK': 0, 'epochs': 48, 'lambda_i': 0.03276387841543247, 'lambda_j': 0.001117568816174994, 'learning_rate': 0.06985386544819226}. Best is trial 62 with value: 0.042585162596466386.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIM_BPR_Python(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

Epoch 1, Iteration 35736 in 10.45 seconds. Samples per second 3420.20
Epoch 2, Iteration 35736 in 7.37 seconds. Samples per second 4849.81
Epoch 3, Iteration 35736 in 5.94 seconds. Samples per second 6016.87
Epoch 4, Iteration 35736 in 5.10 seconds. Samples per second 7009.05
Epoch 5, Iteration 35736 in 4.49 seconds. Samples per second 7951.01
Epoch 6, Iteration 35736 in 4.10 seconds. Samples per second 8718.89
Epoch 7, Iteration 35736 in 3.84 seconds. Samples per second 9317.92
Epoch 8, Iteration 35736 in 3.62 seconds. Samples per second 9858.79
Epoch 9, Iteration 35736 in 3.45 seconds. Samples per second 10362.18
Epoch 10, Iteration 35736 in 3.39 seconds. Samples per second 10542.93
Epoch 11, Iteration 35736 in 3.27 seconds. Samples per second 10936.93
Epoch 12, Iteration 35736 in 3.22 seconds. Samples per second 11111.56
Epoch 13, Iteration 35736 in 3.28 seconds. Samples per second 10906.36
Epoch 14, Iteration 35736 in 3.13 seconds. Samples per second 11414.39
Epoch 15, Iteration 35

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/working/RECsys_Challenge2024/Dataset/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SLIM_BPR_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/best_params_SLIM_BPR_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/history_SLIM_BPR_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/Submission/submission_SLIM_BPR_MAP.csv' updated successfully.
